In [2]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
import itertools 
import pickle
from sklearn.neural_network import MLPClassifier
from fastText import load_model
from sklearn.model_selection import train_test_split
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer

In [217]:
df = pd.read_csv("try.csv")
df

,Organizacion,Sector,Provincia,pcode,Canton,pcode2,Estado
0,Arche noVa,"Agua, Saneamiento e Higiene",Esmeraldas,EC08,Muisne,EC0803,En ejecución
1,Acción Contra el Hambre,Recuperación temprana,Esmeraldas,EC08,Muisne,EC0803,Finalizado
2,Acción Contra el Hambre,Recuperación temprana,Manabí,EC13,Pedernales,EC1317,Finalizado
3,Arche noVa,"Agua, Saneamiento e Higiene",Esmeraldas,EC08,Muisne,EC0803,En ejecución
4,Arche noVa,"Agua, Saneamiento e Higiene",Manabí,EC13,San Vicente,EC1322,En ejecución
5,Arche noVa,"Agua, Saneamiento e Higiene",Manabí,EC13,Jama,EC1320,En ejecución
6,Catholic Relief Services,"Agua, Saneamiento e Higiene",Esmeraldas,EC08,Muisne,EC0803,En ejecución
7,Catholic Relief Services,"Agua, Saneamiento e Higiene",Manabí,EC13,Pedernales,EC1317,En ejecución
8,Catholic Relief Services,"Agua, Saneamiento e Higiene",Manabí,EC13,Jama,EC1320,En ejecución
9,Cooperación Técnica Belga,"Agua, Saneamiento e Higiene",Manabí,EC13,Jipijapa,EC1306,En ejecución


In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruochen99/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
fasttext_model = 'wiki.en.bin'
fmodel = load_model(fasttext_model)

In [6]:
def lower_cols(lst):
    return [word.lower() for word in lst]


def remove_chars(lst):

    cleaned = [re.sub('\s+', ' ', mystring).strip() for mystring in lst]
    cleaned = [re.sub(r'[[^A-Za-z0-9\s]+]', ' ', mystr) for mystr in cleaned]
    cleaned = [mystr.replace('_', ' ') for mystr in cleaned]
    return cleaned

def clean_cols(data):
    data = lower_cols(data)
    data = remove_chars(data)
    return data

In [219]:
def preprocess(pandas_dataset, df_target):
    if (not pandas_dataset.empty):
    	organization = 'HDX'   #Replace if datasets contains organization
    	headers = list(pandas_dataset.columns.values)
    	headers = clean_cols(headers)
    for i in range(len(headers)):
        try:
            dic = {'Header': headers[i], 
                   'Data': list(pandas_dataset.iloc[1:, i]), 
                   'Relative Column Position': (i+1) / len(pandas_dataset.columns), 
                   'Organization': organization,
                   }
            df_target.loc[len(df_target)] = dic
        except:
            raise Exception("Error: arguments not matched")

    df_result = transform_vectorizers(df_target)
    return df_result

def transform_vectorizers(df_target):
    cols = ['Header_embedding', 'Organization_embedded', 'BOW_counts', 'ngrams_counts']
    df = pd.DataFrame(columns = cols)
    long_string = []
    for i in df_target['Data']:
        result_by_tag = word_extract(i)
        holder_list = ''.join(result_by_tag)
        long_string.append(holder_list)
    bag_vectorizer = CountVectorizer()
    corpus = long_string
    X_vecs_bag = bag_vectorizer.fit_transform(corpus)
    df['BOW_counts'] = [item for item in X_vecs_bag.toarray()]

    ngrams = generate_n_grams(df_target['Header'], 3)
    ngrams.append([ngrams[-1][-2], ngrams[-1][-1], 'dummy'])
    ngrams.append([ngrams[-2][-1], 'dummy', 'dummy'])
    ngrams_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    X_vec_grams = ngrams_vectorizer.fit_transform(ngrams)
    tmp = X_vec_grams.toarray()
    for i in range(len(tmp[-1])):
        if tmp[-1][i] == 2:
            dummy = i
    final = []
    for i in range(len(tmp)):
        final.append(np.delete(tmp[i], dummy).tolist())    
    print(final)
#     print(len(final))
    df['ngrams_counts'] = [item for item in final]
    df['Header_embedding'] = df_target['Header'].astype(str).apply(fmodel.get_sentence_vector)
    df['Organization_embedded'] = df_target['Organization'].astype(str).apply(fmodel.get_sentence_vector)
    cols = ['Header_embedding', 'Organization_embedded', 'BOW_counts', 'ngrams_counts']
    df['features_combined'] = df[cols].values.tolist()
#     print(df['features_combined'][4])
#     df['features_combined'] = df['features_combined'].apply(lambda x: np.concatenate(x, axis=None))
    df['features_combined'] = df['features_combined'].apply(lambda x: [val for item in x for val in item])
#     df['features_combined'] = df['features_combined'].apply(lambda x: x[0:300])
    df.dropna()
    return df

In [208]:
def remove_stop_words(data_lst):
    #remove stopwords from the data including 'the', 'and' etc.
    wordsFiltered = []
    for w in data_lst:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

def word_extract(row):
    ignore = ['nan']
    no_white = [i.lstrip() for i in row if i not in ignore and not isinstance(i, float)]
    cleaned_text = [w.lower() for w in no_white if w not in ignore]
    return cleaned_text

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS 

from nltk import ngrams
def generate_n_grams(data_lst, n):
    # cleaned = remove_chars(list(data_lst))
    # cleaned = clean_cols(cleaned)
    cleaned = remove_stop_words(data_lst)
    #make sure that n_grams 'refresh' when a new dataset is encountered!!!!   
    return list(ngrams(cleaned, n))

In [220]:
processed_dataset = preprocess(df, 
                pd.DataFrame(columns=['Header','Data','Relative Column Position','Organization']))

[[0, 0, 1, 0, 0, 1, 1], [0, 0, 0, 1, 0, 1, 1], [1, 0, 0, 1, 0, 1, 0], [1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0]]


In [195]:
len(processed_dataset)

10

In [83]:
processed_dataset['features_combined'][1][626]

0

In [221]:
diff = 2934 - len(processed_dataset['features_combined'][0])
for i in range(len(processed_dataset)):
    for j in range(diff):
        processed_dataset['features_combined'][i].append(0)
        
len(processed_dataset['features_combined'][6])

2934

In [67]:
model = pickle.load(open("model.pkl", "rb"))

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [222]:
model.predict(list(processed_dataset['features_combined']))

array(['affected', 'date', 'affected', 'country', 'affected', 'country',
       'date'],
      dtype='<U12')